# Skin Cancer Detection using Machine Learning and Computer Vision

## Project Description

This project aims to develop a robust and accurate skin cancer detection system using machine learning and computer vision techniques. By leveraging a dataset of skin lesion images, the system will be trained to classify images into different categories of skin cancer, aiding in early diagnosis and treatment. The project involves data preprocessing, feature extraction, model training, and evaluation to ensure high performance and reliability.

In [76]:
# Étape 1 : Installer les bibliothèques nécessaires
%pip install tensorflow
%pip install matplotlib
%pip install numpy

import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import EfficientNetB0 # type: ignore #ingore the warning
from tensorflow.keras.layers import Concatenate, Input, Dense, Dropout, Flatten # type: ignore #ingore the warning
from tensorflow.keras.models import Model # type: ignore #ingore the warning
from tensorflow.keras.optimizers import Adam # type: ignore #ingore the warning
from tensorflow.keras.utils import plot_model # type: ignore #ingore the warning
from tensorflow.keras.preprocessing.image import load_img, img_to_array # type: ignore #ingore the warning
from tensorflow.keras.applications.efficientnet import preprocess_input # type: ignore #ingore the warning



Note: you may need to restart the kernel to use updated packages.



In [ ]:
# Fonction pour charger les matrices Y, Cb, Cr à partir du fichier
def load_ycbcr_from_file(file_path):
    """
    Charger les matrices Y, Cb, Cr à partir d'un fichier texte.
    Les fichiers doivent contenir trois sections avec des matrices pour chaque composante.
    """
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Identifier les sections et extraire les matrices
    y_start = lines.index("Matrice de luminance (Y):\n") + 1
    cb_start = lines.index("Matrice de chrominance (Cb):\n") + 1
    cr_start = lines.index("Matrice de chrominance (Cr):\n") + 1

    y = np.loadtxt(lines[y_start:cb_start - 1], dtype=np.float32)
    cb = np.loadtxt(lines[cb_start:cr_start - 1], dtype=np.float32)
    cr = np.loadtxt(lines[cr_start:], dtype=np.float32)

    return y, cb, cr

In [ ]:
# Fonction pour préparer l'entrée pour le modèle
def prepare_ycbcr_input(y, cb, cr):
    """
    Prépare l'entrée pour le modèle à partir des matrices Y, Cb, Cr.
    """
    # Empiler les canaux Y, Cb, Cr pour former une image (100, 100, 3)
    image = np.stack([y, cb, cr], axis=-1)  # Shape: (100, 100, 3)

    # Normaliser l'image (en utilisant une normalisation simple, pas celle d'ImageNet)
    image = image / 255.0  # Normalisation simple entre 0 et 1

    # Ajouter la dimension du batch (1, 100, 100, 3)
    image = np.expand_dims(image, axis=0)

    return image

In [ ]:
# Taille de l'image d'entrée
IMAGE_SIZE = (100, 100, 3)

# Entrée pour les images (par exemple 100x100x3)
image_input = Input(shape=(100, 100, 3), name="image_input")

# Charger le modèle EfficientNetB0 préentraîné sans la couche de classification finale
base_model = EfficientNetB0(
    include_top=False,      # Exclure la dernière couche de classification
    weights="imagenet",     # Utiliser les poids préentraînés sur ImageNet
    input_shape=(100, 100, 3)  # Taille des images d'entrée
)

# Geler les couches du modèle de base pour conserver les poids préentraînés
base_model.trainable = False

Forme des métadonnées: <KerasTensor shape=(None, 5), dtype=float32, sparse=False, name=metadata_input>


In [ ]:
# Passage des images dans EfficientNet-B0
x = base_model(image_input, training=False)  # Les images passent dans le modèle préentraîné
x = Flatten()(x)  # Aplatir les caractéristiques extraites

# Ajout de couches fully connected pour la classification
x = Dense(128, activation="relu")(x)  # Couche dense
x = Dropout(0.5)(x)  # Dropout pour régularisation
output = Dense(1, activation="sigmoid")(x)  # Sigmoid pour une classification binaire

# Définir le modèle final avec une seule entrée (image)
model = Model(inputs=image_input, outputs=output)

In [130]:
model.compile(
    optimizer=Adam(learning_rate=0.001),  # Optimiseur Adam
    loss="binary_crossentropy",          # Fonction de perte pour la classification binaire
    metrics=["accuracy"]                 # Suivi de la précision
)

# Résumé du modèle
model.summary()

Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_67      │ (None, 100, 100,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ efficientnetb0      │ (None, 4, 4,      │  4,049,571 │ input_layer_67[0… │
│ (Functional)        │ 1280)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ metadata_input      │ (None, 5)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_19          │ (None, 20480)     │          0 │ efficientnetb0[0… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_50 (Dense)    │ (None, 32)        │        192 │ metadata_input[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_15      │ (None, 20512)     │          0 │ flatten_19[0][0], │
│ (Concatenate)       │                   │            │ dense_50[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ image_input         │ (None, 100, 100,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_52 (Dense)    │ (None, 1)         │     20,513 │ concatenate_15[0… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,070,276 (15.53 MB)

 Trainable params: 20,705 (80.88 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [131]:
# Visualisation de l'architecture du modèle
plot_model(model, show_shapes=True, to_file="efficientnetb0_model.png")

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [ ]:
# Fonction de prédiction
def predict_skin_cancer(file_path):
    """
    Prédire à partir des matrices Y, Cb, Cr chargées depuis un fichier.
    """
    # Charger les matrices Y, Cb, Cr à partir du fichier
    y, cb, cr = load_ycbcr_from_file(file_path)

    # Préparer l'entrée pour le modèle
    ycbcr_input = prepare_ycbcr_input(y, cb, cr)


    # Prédiction
    prediction = model.predict(ycbcr_input)

    # Interprétation
    if prediction[0] > 0.5:
        return f"Maligne (probabilité {prediction[0][0]:.2f})"
    else:
        return f"Bénigne (probabilité {1 - prediction[0][0]:.2f})"

# Exemple d'utilisation
file_path = "./DataBase/Matrices/ISIC_0001120_matrices.txt"
result = predict_skin_cancer(file_path)
print(result)

Forme de ycbcr_input_batch: (100, 100, 100, 3)
Forme des métadonnées: (100, 5)


c:\Users\mikai\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['image_input', 'metadata_input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


KeyError: "Exception encountered when calling Functional.call().\n\n\x1b[1m2844619818144\x1b[0m\n\nArguments received by Functional.call():\n  • inputs=('tf.Tensor(shape=(32, 100, 100, 3), dtype=float32)', 'tf.Tensor(shape=(32, 5), dtype=float32)')\n  • training=False\n  • mask=('None', 'None')"